In [0]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import folium

from folium import plugins

In [0]:
%run ./Setup

In [0]:
spark.conf.set("spark.datasource.singlestore.ddlEndpoint", cluster)
spark.conf.set("spark.datasource.singlestore.user", "admin")
spark.conf.set("spark.datasource.singlestore.password", password)
spark.conf.set("spark.datasource.singlestore.disablePushdown", "false")

In [0]:
df1 = (spark.read
       .format("singlestore")
       .load("geo_db.london_connections"))

connections_df = df1.toPandas()

In [0]:
df2 = (spark.read
       .format("singlestore")
       .load("geo_db.london_lines"))

lines_df = df2.toPandas()

In [0]:
df3 = (spark.read
       .format("singlestore")
       .load("geo_db.london_stations"))

stations_df = df3.toPandas()

In [0]:
graph = nx.Graph()

for station_id, station in stations_df.iterrows():
  graph.add_node(station["name"],
                 lon = station["longitude"],
                 lat = station["latitude"],
                 s_id = station["id"])

for connection_id, connection in connections_df.iterrows():
  station1_name = stations_df.loc[stations_df["id"] == connection["station1"], "name"].item()
  station2_name = stations_df.loc[stations_df["id"] == connection["station2"], "name"].item()
  graph.add_edge(station1_name,
                 station2_name,
                 time = connection["time"],
                 line = connection["line"])

In [0]:
len(graph.nodes()), len(graph.edges())

In [0]:
node_positions = {node[0]: (node[1]["lon"], node[1]["lat"]) for node in graph.nodes(data = True)}

In [0]:
dict(list(node_positions.items())[0:5])

In [0]:
edge_lines = [edge[2]["line"] for edge in graph.edges(data = True)]

In [0]:
edge_lines[0:5]

In [0]:
edge_colours = [lines_df.loc[lines_df["line"] == line, "colour"].iloc[0] for line in edge_lines]

In [0]:
edge_colours[0:5]

In [0]:
plt.figure(figsize = (12, 12))
nx.draw(graph,
        pos = node_positions,
        edge_color = edge_colours,
        node_size = 20,
        node_color = "black",
        width = 3)
plt.title("Map of the London Underground", size = 20)
plt.show()

In [0]:
network_df = pd.DataFrame()

lons, lats = map(nx.get_node_attributes, [graph, graph], ["lon", "lat"])
lines, times = map(nx.get_edge_attributes, [graph, graph], ["line", "time"])

for edge in list(graph.edges()):
    network_df = network_df.append(
      {"station_from" : edge[0],
       "lon_from" : lons.get(edge[0]),
       "lat_from" : lats.get(edge[0]),
       "station_to" : edge[1],
       "lon_to" : lons.get(edge[1]),
       "lat_to" : lats.get(edge[1]),
       "line" : lines.get(edge),
       "time" : times.get(edge)
    }, ignore_index = True)

In [0]:
network_df.head()

In [0]:
network_df = pd.merge(network_df, lines_df, how = "left", on = "line")

In [0]:
network_df.head()

In [0]:
London = [51.509865, -0.118092]

m = folium.Map(location = London, tiles = "Stamen Terrain", zoom_start = 12)

for i in range(0, len(stations_df)):
  folium.Marker(
    location = [stations_df.iloc[i]["latitude"], stations_df.iloc[i]["longitude"]],
    popup = stations_df.iloc[i]["name"],
  ).add_to(m)

for i in range(0, len(network_df)):
  folium.PolyLine(
    locations = [(network_df.iloc[i]["lat_from"], network_df.iloc[i]["lon_from"]),
                 (network_df.iloc[i]["lat_to"], network_df.iloc[i]["lon_to"])],
    color = network_df.iloc[i]["colour"],
    weight = 3,
    opacity = 1).add_to(m)

plugins.Fullscreen(
  position = "topright",
  title = "Fullscreen",
  title_cancel = "Exit",
  force_separate_button = True).add_to(m)
m

In [0]:
shortest_path = nx.shortest_path(graph, "Oxford Circus", "Canary Wharf", weight = "time")

In [0]:
shortest_path

In [0]:
shortest_path_df = pd.DataFrame({"name" : shortest_path})

In [0]:
merged_df = pd.merge(shortest_path_df, stations_df, how = "left", on = "name")

In [0]:
merged_df

In [0]:
m = folium.Map(tiles = "Stamen Terrain")

sw = merged_df[["latitude", "longitude"]].min().values.tolist()
ne = merged_df[["latitude", "longitude"]].max().values.tolist()

m.fit_bounds([sw, ne])

for i in range(0, len(merged_df)):
  folium.Marker(
    location = [merged_df.iloc[i]["latitude"], merged_df.iloc[i]["longitude"]],
    popup = merged_df.iloc[i]["name"],
  ).add_to(m)
  
points = tuple(zip(merged_df.latitude, merged_df.longitude))

folium.PolyLine(points, color = "red", weight = 3, opacity = 1).add_to(m)

plugins.Fullscreen(
  position = "topright",
  title = "Fullscreen",
  title_cancel = "Exit",
  force_separate_button = True).add_to(m)
m